In [1]:
import pandas as pd
import os

print("📁 Data Lake Structure:")
print("=" * 50)

# شوف كل الملفات في Silver layer
silver_path = "/home/jovyan/data/silver/"
print(f"Silver Layer: {silver_path}")
for file in os.listdir(silver_path):
    if file.endswith('.parquet'):
        print(f"✅ {file}")

print("\n" + "=" * 50)
print("📊 تحميل البيانات...")
print("=" * 50)

# 1. Traffic Data
traffic_file = "/home/jovyan/data/silver/traffic_clean.parquet"
if os.path.exists(traffic_file):
    traffic_df = pd.read_parquet(traffic_file)
    print(f"🚗 Traffic Data: {traffic_df.shape[0]} صفوف × {traffic_df.shape[1]} أعمدة")
    print(traffic_df.head())
else:
    print(f"❌ {traffic_file} غير موجود")

# 2. Weather Data  
weather_file = "/home/jovyan/data/silver/weather_clean.parquet"
if os.path.exists(weather_file):
    weather_df = pd.read_parquet(weather_file)
    print(f"\n🌤️ Weather Data: {weather_df.shape[0]} صفوف × {weather_df.shape[1]} أعمدة")
    print(weather_df.head())
else:
    print(f"❌ {weather_file} غير موجود")

# 3. Merged Data
merged_file = "/home/jovyan/data/silver/merged_data.parquet"
if os.path.exists(merged_file):
    merged_df = pd.read_parquet(merged_file)
    print(f"\n🔗 Merged Data: {merged_df.shape[0]} صفوف × {merged_df.shape[1]} أعمدة")
    print(merged_df.head())
    print("\n📈 أولى الإحصائيات:")
    print(merged_df.describe())
else:
    print(f"❌ {merged_file} غير موجود")


📁 Data Lake Structure:
Silver Layer: /home/jovyan/data/silver/
✅ merged_data.parquet
✅ weather_clean.parquet
✅ traffic_clean.parquet

📊 تحميل البيانات...
🚗 Traffic Data: 611 صفوف × 10 أعمدة
   traffic_id           date_time    city        area  vehicle_count  \
0      9001.0 2024-01-01 00:00:00  London   Islington         4146.0   
1      9006.0 2024-01-01 05:00:00  London  Kensington         3791.0   
2      9009.0 2024-01-01 08:00:00  London   Islington         2602.0   
3      9015.0 2024-01-01 14:00:00  London      Camden         4940.0   
4      9021.0 2024-01-01 20:00:00  London   Southwark         7839.0   

   avg_speed_kmh  accident_count congestion_level road_condition  visibility_m  
0      35.706890             5.0              Low            Dry        1384.0  
1     105.473608             6.0           Medium          Snowy        7816.0  
2      -7.831573             1.0              Low          Snowy        5523.0  
3      13.529156             9.0              Low    

In [4]:
from minio import Minio
import io
import pandas as pd

# 1️⃣ إنشاء MinIO client (من داخل Docker network نستخدم اسم الخدمة minio)
client = Minio(
    "minio:9002",          # نفس الـ endpoint المستخدمة في python-service
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

bucket_name = "silver"
object_name = "merged_data.parquet"  # غيّره لأي ملف تاني لو حابب

# 2️⃣ تحميل الملف من MinIO في الذاكرة
response = client.get_object(bucket_name, object_name)

# 3️⃣ قراءة المحتوى كـ Parquet إلى DataFrame بدون حفظ على الديسك
data = io.BytesIO(response.read())
df = pd.read_parquet(data)

# 4️⃣ استكشاف البيانات
print(f"✅ Loaded {object_name} from bucket '{bucket_name}'")
print(df.shape)
df.head()


✅ Loaded merged_data.parquet from bucket 'silver'
(4516, 19)


,traffic_id,date_time_traffic,city,area,vehicle_count,avg_speed_kmh,accident_count,congestion_level,road_condition,visibility_m_traffic,weather_id,date_time_weather,season,temperature_c,humidity,rain_mm,wind_speed_kmh,visibility_m_weather,weather_condition
0,9001.0,2024-01-01,London,Islington,4146.0,35.70689,5.0,Low,Dry,1384.0,5001.0,2024-01-01 00:00:00,Winter,6.982986,54.0,44.130140,16.421769,4031.0,Rain
1,9001.0,2024-01-01,London,Islington,4146.0,35.70689,5.0,Low,Dry,1384.0,5003.0,2024-01-01 02:00:00,Winter,11.542604,51.0,11.467968,38.179267,1961.0,Fog
2,9001.0,2024-01-01,London,Islington,4146.0,35.70689,5.0,Low,Dry,1384.0,5010.0,2024-01-01 09:00:00,Winter,-1.347413,94.0,27.201236,69.686114,1968.0,Rain
3,9001.0,2024-01-01,London,Islington,4146.0,35.70689,5.0,Low,Dry,1384.0,5012.0,2024-01-01 11:00:00,Winter,13.482720,94.0,35.940131,47.272753,9312.0,Fog
4,9001.0,2024-01-01,London,Islington,4146.0,35.70689,5.0,Low,Dry,1384.0,5020.0,2024-01-01 19:00:00,Winter,-4.462057,87.0,26.889786,60.036157,3508.0,Rain


In [3]:
!pip install minio pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 1.0 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.3 MB/s eta 0:00:0000:0100:01
